<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/DenseNet169.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login 617d80dc70f383d8ff9ef1c2b94a78ccc8d8db23

     |████████████████████████████████| 1.9 MB 39.1 MB/s 
     |████████████████████████████████| 182 kB 67.9 MB/s 
     |████████████████████████████████| 166 kB 70.9 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 64.2 MB/s 
     |████████████████████████████████| 162 kB 70.7 MB/s 
     |████████████████████████████████| 162 kB 76.9 MB/s 
     |████████████████████████████████| 158 kB 73.9 MB/s 
     |████████████████████████████████| 157 kB 72.4 MB/s 
     |████████████████████████████████| 157 kB 66.7 MB/s 
     |████████████████████████████████| 157 kB 64.9 MB/s 
     |████████████████████████████████| 157 kB 73.7 MB/s 
     |████████████████████████████████| 157 kB 76.7 MB/s 
     |████████████████████████████████| 157 kB 31.8 MB/s 
     |████████████████████████████████| 157 kB 63.8 MB/s 
     |████████████████████████████████| 156 kB 66.6 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [6]:
densenet121 = tf.keras.applications.DenseNet169(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)
model_densenet121=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  densenet121,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
densenet121.trainable=False
model_densenet121.summary()

51877672/51877672 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 densenet169 (Functional)    (None, 1664)              12642880  
                                                                 
 dropout (Dropout)           (None, 1664)              0         
                                                                 
 dense (Dense)               (None, 5)                 8325      
                                                                 
Total params: 12,651,205
Trainable params: 8,325
Non-trainable params: 12,642,880
_________________________________________________________________


In [7]:
model_densenet121.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [8]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "Densenet169"
      )

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [9]:
%%time
epochs=40
history = model_densenet121.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
19/19 [==============================] - ETA: 0s - loss: 1.2640 - accuracy: 0.5287

wandb: Adding directory to artifact (/content/wandb/run-20221106_160911-1a5war4n/files/model-best)... Done. 0.5s


19/19 [==============================] - 224s 8s/step - loss: 1.2640 - accuracy: 0.5287 - val_loss: 0.7599 - val_accuracy: 0.7365
Epoch 2/40
19/19 [==============================] - ETA: 0s - loss: 0.6193 - accuracy: 0.8024

wandb: Adding directory to artifact (/content/wandb/run-20221106_160911-1a5war4n/files/model-best)... Done. 0.4s


...

19/19 [==============================] - 67s 4s/step - loss: 0.0322 - accuracy: 0.9966 - val_loss: 0.0668 - val_accuracy: 0.9865
Epoch 28/40
19/19 [==============================] - ETA: 0s - loss: 0.0285 - accuracy: 0.9966

wandb: Adding directory to artifact (/content/wandb/run-20221106_160911-1a5war4n/files/model-best)... Done. 0.5s


19/19 [==============================] - 67s 4s/step - loss: 0.0285 - accuracy: 0.9966 - val_loss: 0.0664 - val_accuracy: 0.9865
Epoch 29/40
19/19 [==============================] - 4s 194ms/step - loss: 0.0279 - accuracy: 1.0000 - val_loss: 0.0670 - val_accuracy: 0.9865
Epoch 30/40
19/19 [==============================] - ETA: 0s - loss: 0.0250 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221106_160911-1a5war4n/files/model-best)... Done. 0.4s


19/19 [==============================] - 67s 4s/step - loss: 0.0250 - accuracy: 1.0000 - val_loss: 0.0628 - val_accuracy: 0.9865
Epoch 31/40
19/19 [==============================] - 4s 198ms/step - loss: 0.0215 - accuracy: 1.0000 - val_loss: 0.0628 - val_accuracy: 0.9865
Epoch 32/40
19/19 [==============================] - 4s 194ms/step - loss: 0.0212 - accuracy: 1.0000 - val_loss: 0.0647 - val_accuracy: 0.9865
Epoch 33/40
19/19 [==============================] - 4s 193ms/step - loss: 0.0239 - accuracy: 0.9983 - val_loss: 0.0659 - val_accuracy: 0.9865
Epoch 34/40
19/19 [==============================] - 4s 193ms/step - loss: 0.0212 - accuracy: 1.0000 - val_loss: 0.0648 - val_accuracy: 0.9865
Epoch 35/40
19/19 [==============================] - 4s 195ms/step - loss: 0.0195 - accuracy: 1.0000 - val_loss: 0.0638 - val_accuracy: 0.9865
Epoch 36/40
19/19 [==============================] - 4s 195ms/step - loss: 0.0206 - accuracy: 0.9983 - val_loss: 0.0646 - val_accuracy: 0.9865
Epoch 37/40
1

In [10]:
wandb.finish()

accuracy,▁▅▇▇▇███████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▇▇▇██████████████████████████████████
val_loss,█▅▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,1.0
best_epoch,29
best_val_loss,0.06276
epoch,39
loss,0.0165
val_accuracy,0.98649


In [11]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
